In [1]:
!pip install ftfy==6.1.1
!pip install transformers==4.20.1
!pip install tensorflow_addons==0.17.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
import numpy as np
import pickle
import tensorflow as tf
import random
import tensorflow_addons as tfa
import pandas as pd
import warnings,operator
from tqdm import tqdm
import time
from ftfy import fix_text
import os, re, string
import time

warnings.filterwarnings("ignore")
from transformers import logging
from transformers import TFAutoModel, AutoTokenizer
from transformers import RobertaConfig, TFXLMRobertaModel, RobertaTokenizer,TFRobertaModel,TFBertModel
logging.set_verbosity_error()
from keras.preprocessing.sequence import pad_sequences
from google.colab import drive
drive.mount('/content/gdrive')


path_save_model = '/content/gdrive/My Drive/ABSA_English/Model/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
def set_seeds(seed=1):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=1):
    set_seeds(seed=seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'


# Call the above function with seed value
SEED = 42
set_global_determinism(seed=SEED)
tf.keras.utils.set_random_seed(SEED)

In [4]:
import string,re
def clean_doc(text):
  text = text.lower()
  text = text.translate(text.maketrans('', '', string.punctuation))
  text = re.sub(r"[0-9]+", " num ", text)
  text = re.sub('\\s+',' ',text)
  text = text.strip()
  return text

In [5]:
listLabel = "RESTAURANT#GENERAL,SERVICE#GENERAL,FOOD#QUALITY,FOOD#STYLE_OPTIONS,DRINKS#STYLE_OPTIONS,DRINKS#PRICES,RESTAURANT#PRICES,RESTAURANT#MISCELLANEOUS,AMBIENCE#GENERAL,FOOD#PRICES,LOCATION#GENERAL,DRINKS#QUALITY"
categories = listLabel.split(',')

In [6]:
language = "en"
print("Load English model")
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name, do_lower_case=False)

Load English model


In [7]:
# Load RoBertTa model
path_save_model = '/content/gdrive/My Drive/ABSA_English/Model/'

start = time.time()
model = tf.keras.models.load_model(path_save_model + "RoBertTa.h5",custom_objects={"TFRobertaModel": TFRobertaModel})
end = time.time()
print("Time for Loading model: " + str(end - start))

Time for Loading model: 24.56593894958496


In [8]:
# Demo
while(True):
  query = input("Input the query: ")
  start = time.time()
  query_clean = clean_doc(query)
  #print("Query clean: ", query_clean)
  print()
  if query=="STOP":
    break
  encoding_demo = tokenizer([query_clean], return_tensors='tf', padding="max_length", truncation=True, max_length=74)
  demo_ids = encoding_demo['input_ids']
  demo_attention_mask = encoding_demo['attention_mask']
  predicted = model.predict([np.expand_dims(demo_ids[0], axis=0),np.expand_dims(np.array(demo_attention_mask[0]), axis=0)])
  s = ''
  for i, predict in enumerate(predicted):
      index2, value = max(enumerate(predict[0]), key=operator.itemgetter(1))
      if index2 == 1:
          s+= '{' + str(categories[i]) + ', positive}, '
      elif index2 == 2:
          s+= '{' + str(categories[i]) + ', neutral}, '
      elif index2 == 3:
          s+= '{' + str(categories[i]) + ', negative}, '
  print("Output's prediction: ", str(s[:-2]))
  end = time.time()
  print("Time for prediction: " + str(end - start))
  print("="*200)

Input the query: it has great sushi and even better service
Query clean:  it has great sushi and even better service

Output's prediction:  {SERVICE#GENERAL, positive}, {FOOD#QUALITY, positive}
Time for prediction: 10.125906944274902
Input the query: the wine list is wonderful and the food reminds me of my recent trip to italy
Query clean:  the wine list is wonderful and the food reminds me of my recent trip to italy

Output's prediction:  {FOOD#QUALITY, positive}, {DRINKS#STYLE_OPTIONS, positive}
Time for prediction: 0.3439767360687256
Input the query: STOP
Query clean:  stop

